# Fractal Tree

## Background

Fractal Trees are images with a self-repeating structure. In this case the structure is repeated as each branch is a repetition of the initial trunk of the tree. At the end of each branch, two more branches begin, where these branches are smaller versions of the branch they came from. This is done using recursion, which is where a solution to a problem relies on solving smaller instances of the same problem. In this scenario, it is done by calling the function to draw a tree from within itself in order to draw the two new branches. An example of a fractal tree is shown in the image below. 

<img src="images/treeExample.png"/>

## Software Version

The code below uses the Python Image Library (PIL) to create a fractal tree. The two sliders select the recursion depth and spread of the fractal.

In [ ]:
import colorsys
from math import *
from PIL import Image as IM, ImageDraw
from IPython.display import display
from ipywidgets import *

# function to draw tree
# uses recursion to draw trees at end of each branch
def drawTree(x1, y1, angle, depth, maxd, spread, len):
    #check recursion depth, if 0 then does nothing
    if depth > 0:
        # (x1,y1) is start point of branch
        # (x2,y2) is end point of branch
        
        # calculate end point of branch
        x2 = x1 + cos(radians(angle)) * len * depth
        y2 = y1 + sin(radians(angle)) * len * depth
 
        # uses depth/maxDepth for Hue value then converts to RGB 
        # means that each level of branches is a different colour
        r, g, b = colorsys.hsv_to_rgb((float(depth) / maxd) , 1.0, 1.0)
        R, G, B = int(255 * r), int(255 * g), int(255 * b)
 
        # draw branch
        d.line([x1, y1, x2, y2], (R, G, B), depth)
 
        # recursively call function to append 2 new trees to end of branch
        drawTree(x2, y2, angle - spread, depth - 1, maxd, spread, len)
        drawTree(x2, y2, angle + spread, depth - 1, maxd, spread, len)

#function for updating image
#runs when slider values changed
def update(Depth,Spread):
    length = 7
    d = ImageDraw.Draw(img)
    d.rectangle((0, height,width ,0), fill=(0, 0, 0)) #black rectangle to "clear" drawings
    drawTree(width/2,height*0.9,-90,Depth,Depth,Spread,length) #call function to draw tree
    img.save("Tree.png", "PNG") #save image locally
    fractalImage = IM.open('Tree.png') 
    display(fractalImage) #show image in notebook interface

width = 600
height = 480
img = IM.new('RGB', (width, height)) #create new image
d = ImageDraw.Draw(img) #create new drawing object

#set up sliders for interaction
maxDepth = widgets.IntSlider(min=1,max=10,step=1,value=8)
spread = widgets.IntSlider(min=0,max=45,step=1,value=15)
interact(update, Depth = maxDepth, Spread = spread);

## Hardware Version

In order to off-load some processing on to the PL of the PYNQ board, an IP core was made to approximate the sine and cosine of a given angle using CORDIC. Sine and Cosine are used in the drawTree function in order to calculate the endpoint of each branch. CORDIC stands for COordinate Rotation DIgital Computer [1]. It can be used to calculate hyperbolic and trigonometric functions and is well suited to FPGAs as the only operations it requires are addition, subtraction, bitshift and table lookup, and as such works well with fixed point numbers. The IP was developed using the CORDIC 6.0 System Generator block [2].

The IP design is shown below.

<img src="images/cordicsincos.png"/>

This design takes in an angle in radians, and returns 2 values, being the cosine and sine of the angle, respectively.

The simulation ran in Simulink shown above takes an input of 0 and scales it by $2^{13}$ to allow for 13 fractional bits. The correct outputs for $sin(0)$ and $cos(0)$ are 0 and 1, respectively. The IP design gives answers of -0.003174 and 0.9999, which are very close to the exact answer, and as such will be accurate enough for our purposes.

The IP core was added to a Vivado block diagram along with a Zynq 7 PS to make form the block diagram shown below.

<img src="images/blockDiagramTree.png"/>

After validating the design, the HDL wrapper was created in Vivado, before creating the bitstream (.bit), hardware hand-off (.hwh) and Tcl files. These three files were uploaded to the storage of the PYNQ board for use in this project.

### Program

Import overlay class and create new overlay using bitstream file.

In [ ]:
from pynq import Overlay
ol = Overlay("design_tree_wrapper.bit")

The below cell creates a report of the overlay, which shows the CORDIC Sine Cosine algorithm

In [ ]:
ol?

Create simpler alias for IP core

In [ ]:
sin_cos = ol.cordic_sin_cos_0

The below cell defines functions to make it easier to use the IP core, as well as a function for converting unsigned numbers to signed. Inputs are scaled by $2^{13}$ and outputs by $2^{-14}$ to account for fractional parts of the numbers.

In [ ]:
from math import *
def to_signed(val,b):
    signedVal = val-(2**b)*int(str((val)>>b-1))
    return signedVal

def sinPL(angle):
    sin_cos.write(0x00,int((angle*2**(13))))
    sin = to_signed(sin_cos.read(0x08),32)*2**(-14)
    return sin

def cosPL(angle):
    sin_cos.write(0x00,int((angle*2**(13))))
    cos = to_signed(sin_cos.read(0x04),32)*2**(-14)
    return cos


In [ ]:
import colorsys
from math import *
from PIL import Image as IM, ImageDraw
from IPython.display import display
from ipywidgets import *

# function to draw tree
# uses recursion to draw trees at end of each branch
def drawTreePL(x1, y1, angle, depth, maxd, spread, len):
    #check recursion depth, if 0 then does nothing
    if depth > 0:
        # convert angle to radians
        angleRad = radians(angle)
        #use IP core to calculate sine and cosine of angle
        #if structure is a workaround for fact that CORDIC only works
        #for inputs between -180 and 180 degrees
        #not elegant but it does solve the issue
        if angleRad < -(2*pi):
            angleRad = angleRad + 2*pi
        elif angleRad > (2*pi):
            angleRad = angleRad - 2*pi
        if angleRad < -pi:
            #print("Less than -pi",angleRad)
            cosAngle = cosPL(2*pi - abs(angleRad))
            sinAngle = sinPL((-angleRad) - pi)
        elif angleRad > pi:
            #print("more than pi",angleRad)
            cosAngle = cosPL(2*pi- abs(angleRad))
            sinAngle = sinPL(pi - abs(angleRad))
        else:
            #print("no condition",angleRad)
            cosAngle = cosPL(angleRad)
            sinAngle = sinPL(angleRad)            
              
        # length multiplied by depth so branches get shorter towards end of tree
        # (x1,y1) is start point of branch
        # (x2,y2) is end point of branch        
        x2 = x1 + cosAngle * depth * len
        y2 = y1 + sinAngle * depth * len
 
        # uses depth/maxDepth for Hue value then converts to RGB 
        # means that each level of branches is a different colour
        r, g, b = colorsys.hsv_to_rgb((float(depth) / maxd) , 1.0, 1.0)
        R, G, B = int(255 * r), int(255 * g), int(255 * b)
 
        # draw branch
        d.line([x1, y1, x2, y2], (R, G, B), depth)
 
        # recursively call function to append 2 new trees to end of branch
        drawTreePL(x2, y2, angle-spread, depth - 1, maxd, spread, len)
        drawTreePL(x2, y2, angle+spread, depth - 1, maxd, spread, len)

#function for updating image
#runs when slider values changed
def updatePL(Depth,Spread):
    length = 7
    d = ImageDraw.Draw(img)
    d.rectangle((0, height,width ,0), fill=(0, 0, 0)) #black rectangle to "clear" drawings
    drawTreePL(width/2,height*0.9,-90,Depth,Depth,Spread,length) #call function to draw tree
    img.save("TreePL.png", "PNG") #save image locally
    fractalImage = IM.open('TreePL.png') 
    display(fractalImage) #show image in notebook interface

width = 600
height = 480
img = IM.new('RGB', (width, height)) #create new image
d = ImageDraw.Draw(img) #create new drawing object

#set up sliders for interaction
maxDepth = widgets.IntSlider(min=1,max=10,step=1,value=8)
spread = widgets.IntSlider(min=0,max=45,step=1,value=15)
interact(updatePL, Depth = maxDepth, Spread = spread);

Further work to improve the design could be implementing the AXI-Stream interface instead of AXI4-Lite in order to speed up execution. Another possibility could be to implement multi-threading on the PS and parallel computing, for example to compute the next two branches at one time. This could significantly reduce execution time, as well as improving the smoothness and responsiveness of the interactive plot.

## References

[1] J. E. Volder, "The CORDIC Trigonometric Computing Technique," in IRE Transactions on Electronic Computers, vol. EC-8, no. 3, pp. 330-334, Sept. 1959, doi: 10.1109/TEC.1959.5222693.

[2] CORDIC v6.0 LogiCORE IP Product Guide https://www.xilinx.com/support/documentation/ip_documentation/cordic/v6_0/pg105-cordic.pdf